# Train CNN Classifier on human_ocr_ensembl dataset

The dataset comes from the [Genomic Benchmarks](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks). Best reaults achieved are reported in these [tables](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/experiments)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from genomic_benchmarks.data_check import info
import optuna

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Get dataset

In [3]:
info("human_enhancers_cohn", 0)

Dataset `human_enhancers_cohn` has 2 classes: negative, positive.

All lengths of genomic intervals equals 500.

Totally 27791 sequences have been found, 20843 for training and 6948 for testing.


,train,test
negative,10422,3474
positive,10421,3474


In [4]:
dataset = load_dataset("katarinagresova/Genomic_Benchmarks_human_enhancers_cohn")

In [5]:
dataset['train'][0]

{'seq': 'TGGTGGTACTTGTCAGGACTTGGAGCAGCAGGTGCAAGATTTAGTGGGTTGGTTTTAGAATATCTGCTTGGAAAGTGGAAAAACTCAATGGATCATCTAGACTTTGGAATTTATCTCCTTCCCCACTTCTCCACTCCCCCAACAACAACAACAACAACAATGACAACAAAAACACCTGGAATAAACAGGTCATACAACGAGGTAGTTGATAGAATAATGTACTTTCCTTTCAGGCACCCCTTGGAGGAGGCAGATTCTGCCCTTTAAGCTGAATCTGCCTTTCCTGCATTTCCTGAAACTCCTGCATTTCCTGAAATCTTCCTGTATTTTCCTGAAATTTCCTGCCATTCCTGAAACTTTAAGGTAACTGTGTCATTAAAGGAAGGAGAGAAGGGAAGTATTAGGACTGCAGATTTGGGGTGCATGATCAGCCTGGCTCTGAGCTTGCAGACTCCCAGAGTCAGGGAAGGGAGGAGCCACCAGCAACCTTGTGGCTTACT',
 'label': 0}

## Encode and split dataset

In [6]:
def one_hot_encode(sequence, max_length=500):
    one_hot = torch.zeros((4, max_length), dtype=torch.float32)
    
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    
    for i, nucleotide in enumerate(sequence[:max_length]):
        if nucleotide in mapping:
            one_hot[mapping[nucleotide], i] = 1.0

    return one_hot
    
class DNADataset(Dataset):
    def __init__(self, data):
        self.dataset = data
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        seq = self.dataset[idx]['seq']
        label = self.dataset[idx]['label']
        encoded_seq = one_hot_encode(seq)
        return encoded_seq, label

In [7]:
# train data
ds = dataset["train"].with_format("torch")
ds = DNADataset(ds)

# test data
test_ds = dataset["test"].with_format("torch")

train_size = int(0.8 * len(ds))
val_size = len(ds) - train_size

train_ds, val_ds = torch.utils.data.random_split(ds, [train_size, val_size])

test_ds = DNADataset(test_ds)


train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

## Define model

In [8]:
# Define a simple CNN for binary classification of DNA sequences
class DNAClassifierCNN(nn.Module):
    def __init__(self, kernel_size=5, stride = 1):
        super(DNAClassifierCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=kernel_size, stride=stride)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=stride)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=kernel_size, stride=stride)

        self.relu = nn.LeakyReLU()        
        self.fc1 = nn.Linear(self.count_flatten_size(), 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def count_flatten_size(self):
        dummy_input = torch.zeros(1, 4, 500)
        dummy_output = self.pool(self.conv2(self.pool(self.conv1((dummy_input)))))
        return dummy_output.view(-1).size(0)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.reshape(x.size(0), -1)  # Flatten for fully connected layer
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [9]:
# Training loop
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        labels = labels.float().to(DEVICE)
        optimizer.zero_grad()
        
        outputs = model(inputs.to(DEVICE))
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

In [10]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            labels = labels.float().to(DEVICE)
            
            outputs = model(inputs.to(DEVICE))
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
            preds = (outputs.squeeze() > 0.5).float()
            correct += (preds == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    return avg_loss, accuracy

In [12]:
# Run model training and evaluation after each epoch
def evaluation_loop(model, epochs, lr):
    
    adam = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        train_model(model, train_loader, adam, criterion)
        avg_loss, accuracy = evaluate_model(model, val_loader, criterion)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_loss}, Accuracy: {accuracy}')
    
    avg_loss, accuracy = evaluate_model(model, val_loader, criterion)

    print(f'Loss: {avg_loss}, Accuracy: {accuracy}\n')
    
    return accuracy

## Perform training

In [13]:
model = DNAClassifierCNN().to(DEVICE)
evaluation_loop(model, epochs=5, lr=0.001)

Epoch 1/5, Validation Loss: 0.5987304778954455, Accuracy: 0.6699448308946989
Epoch 2/5, Validation Loss: 0.6021936736034073, Accuracy: 0.6778603981770208
Epoch 3/5, Validation Loss: 0.6239265936021586, Accuracy: 0.6685056368433677
Epoch 4/5, Validation Loss: 0.6555210492993129, Accuracy: 0.6541136963300551
Epoch 5/5, Validation Loss: 0.7609890835885783, Accuracy: 0.6114176061405613
Loss: 0.7609890835885783, Accuracy: 0.6114176061405613



0.6114176061405613

## Hyperparam optimization

Let's try to optimize the learning rate, number of training epochs and size of the convolution kernel

In [14]:
best_model = None
best_acc=0

def objective(trial):
    lr = trial.suggest_float('learning_rate', 0.00001, 0.01)
    epochs = trial.suggest_int('epochs', 5, 10)
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    stride = trial.suggest_int('stride', 1, 4)

    print(f"LR: {lr}, Epochs: {epochs}, Kernel size: {kernel_size}, Stride = {stride}")

    model = DNAClassifierCNN(kernel_size=kernel_size, stride=stride).to(DEVICE)
    
    acc = evaluation_loop(model, epochs, lr)
    
    if acc > best_acc:
        best_model = model
    return acc

In [15]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2024-11-23 13:04:53,831] A new study created in memory with name: no-name-f7022d2f-657a-4cd0-a253-8b84b39af3e3


LR: 0.0014032574889357408, Epochs: 5, Kernel size: 3, Stride = 1
Epoch 1/5, Validation Loss: 0.5990545224142438, Accuracy: 0.6761813384504677
Epoch 2/5, Validation Loss: 0.6117415412236716, Accuracy: 0.6627488606380427
Epoch 3/5, Validation Loss: 0.5853131404359833, Accuracy: 0.6862556968097865
Epoch 4/5, Validation Loss: 0.6132927491464688, Accuracy: 0.6735428160230271
Epoch 5/5, Validation Loss: 0.6401779945115097, Accuracy: 0.6553130247061646


[I 2024-11-23 13:09:03,146] Trial 0 finished with value: 0.6553130247061646 and parameters: {'learning_rate': 0.0014032574889357408, 'epochs': 5, 'kernel_size': 3, 'stride': 1}. Best is trial 0 with value: 0.6553130247061646.


Loss: 0.6401779945115097, Accuracy: 0.6553130247061646

LR: 0.003929721323855199, Epochs: 7, Kernel size: 3, Stride = 1
Epoch 1/7, Validation Loss: 0.5959113554190133, Accuracy: 0.6860158311345647
Epoch 2/7, Validation Loss: 0.594130270581209, Accuracy: 0.681458383305349
Epoch 3/7, Validation Loss: 0.6087743146273926, Accuracy: 0.6713840249460302
Epoch 4/7, Validation Loss: 0.6252218964900679, Accuracy: 0.6687455025185896
Epoch 5/7, Validation Loss: 0.7207625132935648, Accuracy: 0.6584312784840489
Epoch 6/7, Validation Loss: 0.8728863951814083, Accuracy: 0.637083233389302
Epoch 7/7, Validation Loss: 1.4222625789751533, Accuracy: 0.6442792036459583


[I 2024-11-23 13:14:49,245] Trial 1 finished with value: 0.6442792036459583 and parameters: {'learning_rate': 0.003929721323855199, 'epochs': 7, 'kernel_size': 3, 'stride': 1}. Best is trial 0 with value: 0.6553130247061646.


...

Loss: 0.6931500407575651, Accuracy: 0.49796114176061407

LR: 0.008066449792986534, Epochs: 8, Kernel size: 3, Stride = 2
Epoch 1/8, Validation Loss: 0.6209878284512586, Accuracy: 0.6442792036459583
Epoch 2/8, Validation Loss: 0.6026420620561556, Accuracy: 0.660110338210602
Epoch 3/8, Validation Loss: 0.6022365022706622, Accuracy: 0.673782681698249
Epoch 4/8, Validation Loss: 0.6115119291170863, Accuracy: 0.6521947709282802
Epoch 6/8, Validation Loss: 0.6031349078389524, Accuracy: 0.681458383305349
Epoch 7/8, Validation Loss: 0.5968186272919633, Accuracy: 0.6826577116814584
Epoch 8/8, Validation Loss: 0.5880206056678569, Accuracy: 0.6800191892540177


[I 2024-11-23 13:27:54,114] Trial 3 finished with value: 0.6800191892540177 and parameters: {'learning_rate': 0.008066449792986534, 'epochs': 8, 'kernel_size': 3, 'stride': 2}. Best is trial 3 with value: 0.6800191892540177.


Loss: 0.5880206056678569, Accuracy: 0.6800191892540177

LR: 0.007786720650814143, Epochs: 9, Kernel size: 5, Stride = 2
Epoch 1/9, Validation Loss: 0.5956250106105367, Accuracy: 0.6766610698009115
Epoch 2/9, Validation Loss: 0.5839012494979013, Accuracy: 0.6886543535620053
Epoch 3/9, Validation Loss: 0.604105435027421, Accuracy: 0.6797793235787959
Epoch 4/9, Validation Loss: 0.5918430799746331, Accuracy: 0.6888942192372272
Epoch 5/9, Validation Loss: 0.5641004827641348, Accuracy: 0.7100023986567522
Epoch 6/9, Validation Loss: 0.5844512979947883, Accuracy: 0.6831374430319022
Epoch 7/9, Validation Loss: 0.5670674966491815, Accuracy: 0.7006476373230991
Epoch 8/9, Validation Loss: 0.5871302228392535, Accuracy: 0.6857759654593427
Epoch 9/9, Validation Loss: 0.5743113435406721, Accuracy: 0.6956104581434397


[I 2024-11-23 13:35:15,148] Trial 4 finished with value: 0.6956104581434397 and parameters: {'learning_rate': 0.007786720650814143, 'epochs': 9, 'kernel_size': 5, 'stride': 2}. Best is trial 4 with value: 0.6956104581434397.


Loss: 0.5743113435406721, Accuracy: 0.6956104581434397



In [16]:
print(f"Best hyperparameters: {study.best_params}")
print(f"Best value (validation AU PRC): {study.best_value}")

Best hyperparameters: {'learning_rate': 0.007786720650814143, 'epochs': 9, 'kernel_size': 5, 'stride': 2}
Best value (validation AU PRC): 0.6956104581434397


In [ ]:
# Perform Testing

In [17]:
best_params = study.best_params

best_model = DNAClassifierCNN(kernel_size=best_params['kernel_size'], stride=best_params['stride']).to(DEVICE)

optimizer = optim.AdamW(best_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.BCELoss()

for epoch in range(best_params['epochs']):
    train_model(best_model, train_loader, optimizer, criterion)

test_loss, test_accuracy = evaluate_model(best_model, test_loader, criterion)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 0.6932481705048762
Test Accuracy: 0.5
